# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта №4). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — она уже сделана.

Нужно построить модель с максимально большим значением *accuracy* (по крайней мере 0.75) на обучающей выборке.

## Изучение файла

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('users_behavior.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
data.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Частично просмотрев таблицу, а также воспользовавшись `pandas.DataFrame.info()`, мы убедились, что предобработка данных действительно уже была произведена. Данные изменять не нужно.

## Разбивка данных на выборки

In [5]:
data_train, data_valid_and_test = train_test_split(data, test_size=0.4, random_state=999)

In [6]:
data_train.head()

,calls,minutes,messages,mb_used,is_ultra
392,55.0,413.69,39.0,14360.59,0
2701,43.0,173.90,60.0,23233.82,1
2156,1.0,6.96,56.0,17956.44,0
2779,49.0,255.32,0.0,8413.88,0
1941,189.0,1188.18,172.0,25142.93,1


Сначала разобьём выборку как 60% тренировочных данных и 40% валидационных и тестовых.

In [7]:
data_valid, data_test = train_test_split(data_valid_and_test, test_size=0.5, random_state=999)

In [8]:
data_valid.head()

,calls,minutes,messages,mb_used,is_ultra
1004,37.0,279.41,0.0,27548.01,1
2271,56.0,454.16,60.0,11953.02,0
180,99.0,634.32,43.0,16436.32,0
1623,23.0,169.25,20.0,7476.26,0
1961,57.0,372.11,12.0,18254.09,0


In [9]:
data_test.head()

,calls,minutes,messages,mb_used,is_ultra
2368,82.0,539.18,46.0,16021.77,0
184,73.0,576.62,42.0,7865.58,0
2920,48.0,333.11,91.0,13643.08,0
950,65.0,421.87,28.0,23488.98,0
1999,56.0,398.45,4.0,23682.94,0


После чего эти 40 процентов поделим пополам.

## Исследование моделей

### Получение признаков и целевого признака

In [10]:
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

In [11]:
features_valid = data_valid.drop('is_ultra', axis=1)
target_valid = data_valid['is_ultra']

In [12]:
features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

Успешно получили признаки и целевой признак.

### Модель логистической регрессии

In [21]:
param_grid = {'max_iter': range(101, 1001, 25)}

model_logreg = LogisticRegression(random_state=999, solver='lbfgs')
grid_search = GridSearchCV(model_logreg, param_grid, scoring='accuracy', cv=5)
grid_search.fit(features_train, target_train)

best_model_logreg = grid_search.best_estimator_
best_result_logreg = grid_search.best_score_
best_iters = best_model_logreg.max_iter

print("Accuracy лучшей модели на валидационной выборке:", best_result_logreg)
print("Iters лучшей модели на валидационной выборке:", best_iters)

Accuracy лучшей модели на валидационной выборке: 0.7334136329991251
Iters лучшей модели на валидационной выборке: 101


К сожалению, эта модель не достигла порога в 75%. Возможно, эти данные не совсем подходят для логической регрессии.

### Модель дерева

In [14]:
param_grid = {'max_depth': range(1, 16)}

model_tree = DecisionTreeClassifier(random_state=999)
grid_search = GridSearchCV(model_tree, param_grid, scoring='accuracy', cv=5)
grid_search.fit(features_train, target_train)

best_model_tree = grid_search.best_estimator_
best_result_tree = grid_search.best_score_
best_depth = best_model_tree.max_depth

print("Accuracy лучшей модели на валидационной выборке:", best_result_tree)
print("Depth лучшей модели на валидационной выборке:", best_depth)

Accuracy лучшей модели на валидационной выборке: 0.7821532871273804
Depth лучшей модели на валидационной выборке: 3


Обычное дерево показывает хорошие результаты (~78.2%), при этом глубина дерева не очень высока относительно максимально допустимого значения.

### Модель леса деревьев

In [15]:
param_grid = {'n_estimators': range(10, 101, 10), 'max_depth': range(1, 16)}

model_forest = RandomForestClassifier(random_state=999)
grid_search = GridSearchCV(model_forest, param_grid, scoring='accuracy', cv=5)
grid_search.fit(features_train, target_train)

best_model_forest = grid_search.best_estimator_
best_result_forest = grid_search.best_score_
best_est = best_model_forest.n_estimators
best_depth_forest = best_model_forest.max_depth

print("Accuracy лучшей модели на валидационной выборке:", best_result_forest)
print("Depth лучшей модели на валидационной выборке:", best_depth_forest)
print("Est лучшей модели на валидационной выборке:", best_est)

Accuracy лучшей модели на валидационной выборке: 0.8075688042527421
Depth лучшей модели на валидационной выборке: 10
Est лучшей модели на валидационной выборке: 100


Лес показал лучший результат в 80%. Скорость модели оставляет желать лучшего, но это, конечно же, объясняется большим количеством деревьев и их глубиной.

Так как модель леса показала лучший результат, проверим её на тестовой выборке.

## Проверка модели на тестовой выборке

### Проверка модели леса

In [22]:
model_forest_test = RandomForestClassifier(random_state=999, n_estimators=100, max_depth=10)
model_forest_test.fit(features_train, target_train)
predictions_test_forest = model_forest_test.predict(features_test)
result_forest_test = model_forest_test.score(features_test, target_test)
print(f"Accuracy модели на тестовой выборке: {result_forest_test}")

Accuracy модели на тестовой выборке: 0.8351477449455676


Точность модели леса на тестовой выборке составила ~83.5%, впечатляюще.

## Проверка модели на адекватность

In [17]:
ultra_count = data.groupby('is_ultra')['is_ultra'].count()
random_accuracy = ultra_count[0] / (ultra_count[0] + ultra_count[1])
random_accuracy

0.693528313627878

Если бы модель в 100% процентах случаях выставляла значение `1` в столбец `is_ultra`, то она была бы точна в 69% случаях. Будем считать, что модель прошла проверку на адекватность.

## Итог

Модели `DecisionTreeClassifier` и `RandomForestClassifier` успешно обучились и справились с поставленной задачей, а вот `LogisticRegression` не достигла порога точности.